In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

In [4]:
X_full = pd.read_csv('train.csv', index_col='Id')
#X_test_full = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv', index_col='Id')

print(X_full.shape)

(1460, 80)


In [5]:
# Separate target from predictors
y = X_full.SalePrice
X = X_full.drop(['SalePrice'], axis=1)

In [6]:
categorical_cols = ['Street']
numerical_cols = ['LotArea', '1stFlrSF','YearBuilt']
my_cols = categorical_cols + numerical_cols
X = X_full[my_cols].copy()
X.head()

,Street,LotArea,1stFlrSF,YearBuilt
Id,,,,
1,Pave,8450,856,2003
2,Pave,9600,1262,1976
3,Pave,11250,920,2001
4,Pave,9550,961,1915
5,Pave,14260,1145,2000


In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder


# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='mean')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [8]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

model = RandomForestRegressor(n_estimators=100, random_state=0)

# Bundle preprocessing and modeling code in a pipeline
house_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Multiply by -1 since sklearn calculates *negative* MAE
scores = -1 * cross_val_score(house_pipeline, X, y,
                              cv=5,
                              scoring='neg_mean_absolute_error')

print("MAE scores:\n", scores)

house_pipeline.fit(X, y)

MAE scores:
 [29883.21178898 29827.87369145 27923.68862052 29052.04083431
 31841.96934524]


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', SimpleImputer(),
                                                  ['LotArea', '1stFlrSF',
                                                   'YearBuilt']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='unknown',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Street'])])),
                ('model', RandomForestRegressor(random_state=0))])

In [9]:
prediction = house_pipeline.predict(
    pd.DataFrame(
        {'Street': ['Pave'], 'LotArea': [9000], '1stFlrSF': [1000], 'YearBuilt': [1980]}
    )
)

print(prediction)

[139435.6]


In [10]:
import pickle

with open("house_pipeline.pickle", "wb") as outfile:
    pickle.dump(house_pipeline, outfile)